In [1]:
import nltk

In [2]:
import jieba
import tflearn

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
import tensorflow as tf

import pickle
with open('middleresult/en_ch_35word.pkl','wb') as whdl:
    pickle.dump((
        train_x,
        test_x,
        train_y,
        test_y,
        ind2ch,
        ch2ind,
        ind2en,
        en2ind,
    ),whdl)

In [4]:
import pickle
with open('middleresult/tokenlizer_output_60000ch_60000en_40words_token.pkl','rb') as fhdl:
    (
         ind2ch,
         ch2ind,
         ind2en,
         en2ind,
         train_x,
         train_y,
    ) = pickle.load(fhdl)

In [5]:
len(ind2ch),len(ind2en)

(8823, 60000)

In [21]:
src_vocab_size = 50003#len(ind2en) + 3
target_vocat_size = 8826#len(ind2ch) + 3
attention_hidden_size = 1024
attention_output_size = 1024
embedding_size = 1024
seq_max_len = 40
num_units = 1024
batch_size = 64
layer_number = 2
max_grad = 1.0
dropout = 0.2

In [22]:
len(train_x),len(train_y)

(1905944, 1905944)

In [8]:
#train_x = [i[::-1] for i in train_x]

In [9]:
train_x = tf.contrib.keras.preprocessing.sequence.pad_sequences(train_x,seq_max_len,padding='post')
train_y = tf.contrib.keras.preprocessing.sequence.pad_sequences(train_y,seq_max_len,padding='post')

In [10]:
import random
index = random.randint(0,len(train_x))
print(' '.join([ind2en.get(i,'') for i in train_x[index]]))
print(' '.join([ind2ch.get(i,'') for i in train_y[index]]))

they wanted to see if people would share the video using bluetooth wireless technology .                         
他 们 想 看 看 ， 人 们 是 否 会 使 用 蓝 牙 无 线 技 术 分 享 这 个 视 频 。              


In [11]:
train_x.shape,train_y.shape

((1925196, 40), (1925196, 40))

In [12]:
from sklearn.cross_validation import train_test_split

In [13]:
train_x,test_x,train_y,test_y = train_test_split(train_x,train_y , test_size=0.01, random_state=42)

In [14]:
train_x[train_x >= src_vocab_size] = 1 
test_x[test_x >= src_vocab_size] = 1 

In [15]:
train_y[train_y >= target_vocat_size] = 1 
test_y[test_y >= target_vocat_size] = 1 

In [16]:
import numpy as np
x_last_index = np.max(train_x)
y_last_index = np.max(train_y)

ind2en[x_last_index] = '<go>'
ind2ch[y_last_index] = '<go>'
en2ind['<go>'] = x_last_index
ch2ind['<go>'] = y_last_index

ind2en[1] = '<unk>'
ind2ch[1] = '<unk>'
en2ind['<unk>'] = 1
ch2ind['<unk>'] = 1

In [23]:
import random
for i in range(5):
    index = random.randint(0,len(train_x))
    print(' '.join([ind2en.get(i,'') for i in train_x[index]]))
    print(' '.join([ind2ch.get(i,'') for i in train_y[index]]))

the most recent , the permian , was thought to have been an impact                          
最 近 一 次 ， 也 就 是 二 叠 纪 生 物 灭 绝 事 件 被 认 为 是 撞 击 的 结 果 ，             
a vulnerability in the default installation of the pl / sql package allows attackers to perform cross - site scripting attacks using the <unk> . print function .            
l 包 中 存 在 一 个 漏 洞 ， 使 得 攻 击 者 能 够 使 用 h t p . p r i n t 函 数 执 行 跨 站 点 脚 本 攻 击 。
proponents argue that it is wrong to force anyone to contribute money to a union .                        
这 也 是 美 国 传 统 工 业 带 第 一 个 通 过 此 法 的 州 。                    
many people dream about living on an island in the south seas .                           
许 多 人 向 往 在 南 海 的 一 个 岛 上 生 活 。                        
( b ) instead of ordering the grant of a licence to the applicant , order the existing licence to be amended .                 
( b ) 可 命 令 将 现 有 的 特 许 修 订 ， 而 非 命 令 将 一 项 特 许 批 予 该 申 请 人 。         


In [24]:
from tensorflow.python.layers import core as layers_core


In [25]:
import tensorflow as tf
import tflearn
tf.reset_default_graph()
config = tf.ConfigProto(log_device_placement=True,allow_soft_placement = True)
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.Session(config=config)


with tf.device('/gpu:0'):
    initializer = tf.random_uniform_initializer(
        -0.08, 0.08)
    tf.get_variable_scope().set_initializer(initializer)
    
    x = tf.placeholder("int32", [None, None])
    y = tf.placeholder("int32", [None, None])
    y_in = tf.placeholder("int32",[None,None])
    x_len = tf.placeholder("int32",[None])
    y_len = tf.placeholder("int32",[None])
    x_real_len = tf.placeholder("int32",[None])
    y_real_len = tf.placeholder("int32",[None])
    learning_rate = tf.placeholder(tf.float32, shape=[])
    
    # embedding
    embedding_encoder = tf.get_variable(
        "embedding_encoder", [src_vocab_size, embedding_size],dtype=tf.float32)
    embedding_decoder = tf.get_variable(
        "embedding_decoder", [target_vocat_size, embedding_size],dtype=tf.float32)
    
    encoder_emb_inp = tf.nn.embedding_lookup(
        embedding_encoder, x)
    decoder_emb_inp = tf.nn.embedding_lookup(
        embedding_decoder, y_in)
    
    # encoder
    num_bi_layers = int(layer_number / 2)
    cell_list = []
    for i in range(num_bi_layers):
        cell_list.append(
            tf.contrib.rnn.DropoutWrapper(
                tf.contrib.rnn.BasicLSTMCell(num_units), input_keep_prob=(1.0 - dropout)
            )
        )
    if len(cell_list) == 1:
        encoder_cell = cell_list[0]
    else:
        encoder_cell = tf.contrib.rnn.MultiRNNCell(cell_list)
        
    cell_list = []
    
    for i in range(num_bi_layers):
        cell_list.append(
            tf.contrib.rnn.DropoutWrapper(
                tf.contrib.rnn.BasicLSTMCell(num_units), input_keep_prob=(1.0 - dropout)
            )
        )
    if len(cell_list) == 1:
        encoder_backword_cell = cell_list[0]
    else:
        encoder_backword_cell = tf.contrib.rnn.MultiRNNCell(cell_list)
    
    bi_outputs, bi_encoder_state = tf.nn.bidirectional_dynamic_rnn(
        encoder_cell,encoder_backword_cell, encoder_emb_inp,
        sequence_length=x_len,dtype=tf.float32)
    encoder_outputs = tf.concat(bi_outputs, -1)
    
    if num_bi_layers == 1:
        encoder_state = bi_encoder_state
    else:
        encoder_state = []
        for layer_id in range(num_bi_layers):
            encoder_state.append(bi_encoder_state[0][layer_id])  # forward
            encoder_state.append(bi_encoder_state[1][layer_id])  # backward
        encoder_state = tuple(encoder_state)
    
    # decoder 
    #decoder_cell = tf.contrib.rnn.BasicLSTMCell(num_units)
    cell_list = []
    for i in range(layer_number):
        cell_list.append(
            tf.contrib.rnn.DropoutWrapper(
                tf.contrib.rnn.BasicLSTMCell(num_units), input_keep_prob=(1.0 - dropout)
            )
        )
    if len(cell_list) == 1:
        decoder_cell = cell_list[0]
    else:
        decoder_cell = tf.contrib.rnn.MultiRNNCell(cell_list)
    
    # Helper
    
    # attention
    attention_mechanism = tf.contrib.seq2seq.LuongAttention(
        attention_hidden_size, encoder_outputs,
        memory_sequence_length=x_real_len,scale=True)
    decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
        decoder_cell, attention_mechanism,
        attention_layer_size=attention_output_size)
    
    
    projection_layer = layers_core.Dense(
        target_vocat_size, use_bias=False)
    
    
    
    # Dynamic decoding
    with tf.variable_scope("decode_layer"):
        helper = tf.contrib.seq2seq.TrainingHelper(
            decoder_emb_inp,sequence_length= y_len)
        decoder = tf.contrib.seq2seq.BasicDecoder(
            decoder_cell, helper, initial_state = decoder_cell.zero_state(dtype=tf.float32,batch_size=batch_size),
            output_layer=projection_layer)
       
        outputs, _,___  = tf.contrib.seq2seq.dynamic_decode(decoder)
        logits = outputs.rnn_output

        target_weights = tf.sequence_mask(
            y_real_len, seq_max_len, dtype=logits.dtype)
    
    # predicting
    # Helper
    with tf.variable_scope("decode_layer", reuse=True):
        helper_predict = tf.contrib.seq2seq.GreedyEmbeddingHelper(
            embedding_decoder,
            tf.fill([batch_size], ch2ind['<go>']), 0)
        decoder_predict = tf.contrib.seq2seq.BasicDecoder(
            decoder_cell, helper_predict, initial_state = decoder_cell.zero_state(dtype=tf.float32,batch_size=batch_size),
            output_layer=projection_layer)
        outputs_predict,_, __ = tf.contrib.seq2seq.dynamic_decode(
            decoder_predict, maximum_iterations=test_y.shape[1] * 2)
    translations = outputs_predict.sample_id

    # calculate loss
    crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=y, logits=logits)
    train_loss = (tf.reduce_sum(crossent * target_weights) /
        batch_size)
    
    optimizer_ori = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    trainable_params = tf.trainable_variables()
    gradients = tf.gradients(train_loss, trainable_params)
    clip_gradients, _ = tf.clip_by_global_norm(gradients, max_grad)
    global_step = tf.Variable(0, trainable=False, name='global_step')
    optimizer = optimizer_ori.apply_gradients(
            zip(clip_gradients, trainable_params), global_step=global_step)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(train_loss)
    #trainop = tflearn.TrainOp(loss=train_loss, optimizer=optimizer,
    #                          metric=train_loss, batch_size=64)


In [26]:
def cal_acc(logits,target):
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target[:,:seq_max_len], logits[:,:seq_max_len]))


In [27]:
session.run(tf.global_variables_initializer())

In [28]:
saver = tf.train.Saver()

In [29]:
#saver.restore(session,'middleresult/align/result_1_20847')

In [31]:
saver.save(session,'middleresult/result_deep')

'middleresult/result_deep'

In [40]:
#saver.save(session,'middleresult/result_char')

'middleresult/result_char'

In [32]:
from utils import Dataset,ProgressBar

In [33]:
from utils import *
train_set = Dataset(train_x,train_y)
test_set = Dataset(test_x,test_y)

In [34]:
def get_bleu_score(predict,target):
    try:
        target = [[[j for index,j in enumerate(i) if j > 0 or index < 4]] for i in target]
        predict = [[j for index,j in enumerate(i) if j > 0 or index < 4] for i in predict]
        BLEUscore = nltk.translate.bleu_score.corpus_bleu(target,predict)
    except:
        BLEUscore = -1
    return BLEUscore

In [35]:
import numpy as np
def calc_test_loss(test_set = Dataset(test_x,test_y),display=True):
    accs = []
    worksum = int(len(test_x) / batch_size)
    loss_list = []
    predict_list = []
    target_list = []
    source_list = []
    pb = ProgressBar(worksum=worksum,info="validating...",auto_display=display)
    pb.startjob()
    #test_set = Dataset(test_x,test_y)
    for j in range(worksum):
        batch_x,batch_y = test_set.next_batch(batch_size)
        lx = [seq_max_len] * batch_size
        ly = [seq_max_len] * batch_size
        bx = [np.sum(m > 0) for m in batch_x]
        by = [np.sum(m > 0) for m in batch_y]
        tmp_loss,tran = session.run([train_loss,translations],feed_dict={x:batch_x,y:batch_y,
                                                     y_in:
                                                     np.concatenate((
                                                     np.ones((batch_y.shape[0],1),dtype=np.int) * ch2ind['<go>'],batch_y[:,:-1]) ,axis=1)
                                                     ,x_len:lx,y_len:ly,
                                                                        y_real_len:by,
                                                                        x_real_len:bx})
        loss_list.append(tmp_loss)
        tmp_acc = cal_acc(tran,batch_y)
        accs.append(tmp_acc)
        predict_list += [i for i in tran]
        target_list += [i for i in batch_y]
        source_list += [i for i in batch_x]
        pb.complete(1)
    return np.average(loss_list),np.average(accs),get_bleu_score(predict_list,target_list),predict_list,target_list,source_list

In [36]:
w_loss,w_acc,bleu_score,predict_list,target_list,source_list = calc_test_loss(Dataset(train_x[::100],train_y[::100]))

validating... 100.00 % [==================================================>] 300/300 	 used:190s eta:0 s

In [37]:
w_loss,w_acc,bleu_score

(239.4696, 6.9010416666666669e-05, 0.005684131788259892)

In [38]:
def get_all_text(x):
    return [' '.join([ind2ch.get(j,'') for j in i]) for i in x]
def get_all_en_text(x):
    return [' '.join([ind2en.get(j,'') for j in i]) for i in x]

In [39]:
texts = get_all_text(predict_list)
texts[:10]

['畑 畑 畑 畑 意 意 键 啜 啜 键 啜 啜 啜 啜 笾 笾 敤 想 炫 炫 炫 徼 徼 徼 徼 徼 徼 綫 綫 嵬 嵬 嵬 嵬 嵬 嵬 嵬 當 當 當 當 挽 當 骓 當 \ue6dd 蜇 蜇 蜇 蜇 蜇 蜇 蜇 蜇 蜇 蜇 蜇 蜇 蜇 蜇 瑟 秤 秤 σ 由 秤 秤 旎 旎 秤 秤 秤 募 募 單 \ue078 單 單 單 素 素',
 'ｈ 畑 ｈ ｈ 炫 炫 炫 炫 炫 炫 炫 炫 狈 狈 狈 狈 狈 狈 狈 狈 狈 狈 狈 狈 們 鼽 赂 赂 赂 赂 赂 赂 赂 赂 赂 赂 赂 赂 赂 赂 赂 茕 赂 弥 弥 赂 赂 赂 赂 赂 齧 敘 镗 镗 镗 镗 镗 镗 镗 却 灭 镗 镗 ⒚ 镗 墟 墟 墟 吾 撒 撒 撒 線 吾 蹲 摞 哓 糜 哓 验',
 '迨 痷 畑 畑 禜 禜 禜 禜 禜 蕨 蕨 蕨 莨 檀 檀 莨 檀 蜇 蜇 蜇 绪 绪 $ 绪 绪 绪 绪 御 御 御 卖 御 御 御 劫 槲 槲 槲 劫 劫 劫 劫 劫 劫 劫 劫 劫 劫 劫 劫 劫 环 环 矛 溫 环 ㄚ 矛 淨 腊 淨 淨 伱 骑 め 骑 噁 骑 伱 伱 伱 伱 蛛 蛛 伱 罢 罢 罢 罢 怩',
 '颗 母 水 栭 \ue367 \ue367 釜 釜 逻 \ue367 \ue367 黛 滁 滁 黛 滁 挥 栭 挥 允 允 允 瑕 瑕 瑕 瑕 瑕 瑕 楤 瑕 瑕 瑕 瑕 瑕 瑕 瑕 瑕 仠 仠 仠 仠 \ue710 \ue710 嘎 瘸 嘎 嘎 嘎 嘎 缥 嘎 嘎 嘎 实 嘎 实 嘎 实 实 实 实 陪 栭 栭 栭 陪 苣 苣 苣 苣 苣 苣 為 僅 為 為 為 為 為 為',
 '可 可 可 可 萍 萍 萍 煤 煤 煤 煤 煤 煤 幡 幡 幡 幡 缱 缱 缱 幡 矛 警 笵 蜥 蜥 蜥 幼 幼 幼 幡 幼 幼 幼 幼 す 矬 枫 す 枫 枫 枫 枫 莸 莸 莸 枫 枫 枫 枫 枫 枫 枫 枫 枫 枫 枫 枫 枫 槲 沘 搡 搡 搡 搡 雜 煤 女 靺 誉 煺 煺 誉 煺 煺 ｉ 豁 豁 豁 惇',
 '畑 畑 畑 畑 畑 畑 畑 稿 炫 庐 庐 庐 庐 庐 雎 社 社 意 社 社 社 社 社 作 作 胸 胸 坫 坫 坫 作 當 當 當 剀 當 蜇 蜇 5 蜇 雎 5 5 5 5 5 \ue60b \ue60b \ue60b \ue60b 雎 \ue60b \ue60b 狞 

In [40]:
texts = get_all_text(target_list)
texts[:10]

['如 果 没 有 其 他 的 事 ， 在 他 的 回 忆 录 里 关 于 阿 尔 巴 尼 亚 的 译 文 会 使 其 成 为 一 本 畅 销 书 。   ',
 '我 们 要 设 法 削 减 本 年 度 的 开 支 。                          ',
 '( 3 ) 根 据 第 ( 1 ) 款 发 出 的 命 令 须 由 仲 裁 人 签 署 ， 并 须 送 交 各 当 事 人 及 交 书 记 存 档 。  ',
 '人 民 解 放 军 ， 在 戒 严 指 挥 机 构 的 统 一 部 署 下 ， 由 中 央 军 事 委 员 会 指 定 的 军 事 机 关 实 施 指 挥 。',
 '金 融 交 易 激 增 之 效 用 的 政 策 制 定 者 ， 这 种 激 增 除 了 为 银 行 家 带 来 丰 厚 报 酬 ， 再 无 其 它 效 用 。',
 '分 析 了 运 移 参 数 的 变 化 对 g c l 中 污 染 物 运 移 的 影 响 。                ',
 '溶 性 多 糖 （ w s p c ） 通 过 修 复 自 身 免 疫 损 伤 ， 促 进 胰 岛 素 分 泌 ， 有 显 著 的 降 低 血 糖 作 用 。',
 '在 某 种 程 度 上 还 标 识 着 我 们 对 于 宇 宙 的 的 物 理 认 知 的 崩 溃 。              ',
 '( 4 ) 在 本 部 适 用 的 任 何 船 舶 上 ， 不 得 在 限 界 线 之 下 的 船 壳 安 装 自 动 通 风 舷 窗 。     ',
 '1 9 7 0 年 左 右 ， 人 们 发 现 特 化 的 不 是 一 块 区 域                   ']

In [41]:
#loss,tran = session.run([train_loss,translations],feed_dict={x:batch_x,y:batch_y,x_len:lx,y_len:ly,learning_rate:lr,y_in:
#                                                                np.concatenate((
#                                                                np.ones((batch_y.shape[0],1),dtype=np.int) * ch2ind['<go>'],batch_y[:,:-1]) ,axis=1)
#                                                               })

In [42]:
#tran.shape
i_save = 0
j_save = 0

In [43]:
print(i_save,j_save)

0 0


In [44]:
model_path = 'align_char_deep'

In [45]:
os.mkdir('middleresult/{}'.format(model_path))
os.mkdir('eval/{}'.format(model_path))

In [ ]:
n_epoch = 60
restore = True
lr = 1
for i in range(i_save,n_epoch):
    
    i_save = i
    worksum = int(len(train_y)/batch_size)
    pb = ProgressBar(worksum=worksum)
    pb.startjob()
    train_loss_list = []
    train_acc_list = []
    for j in range(worksum):
        if restore == True and j < j_save:
            pb.finishsum += 1
            continue
        restore = False
        
        j_save = j
        batch_x,batch_y = train_set.next_batch(batch_size)
        lx = [seq_max_len] * batch_size
        ly = [seq_max_len] * batch_size
        bx = [np.sum(m > 0) for m in batch_x]
        by = [np.sum(m > 0) for m in batch_y]
        by =[m + 2  if m < seq_max_len - 1 else m for m in by ]
        _, loss = session.run([optimizer,train_loss],feed_dict={x:batch_x,y:batch_y,x_len:lx,y_len:ly,learning_rate:lr,y_in:
                                                                np.concatenate((
                                                                np.ones((batch_y.shape[0],1),dtype=np.int) * ch2ind['<go>'],batch_y[:,:-1]) ,axis=1)
                                                                ,y_real_len:by,
                                                                x_real_len:bx
                                                               })
        train_loss_list.append(loss)
        #tmp_train_acc = cal_acc(tran,batch_y)
        #train_acc_list.append(tmp_train_acc)
        pb.info = "iter {} loss:{} lr:{}".format(i + 1,loss,lr)
        val_step = int(worksum / 4)
        if j % val_step == 0 and j != 0:
            test_loss,test_acc,bleu_score,predict_list,target_list,source_list = calc_test_loss()
            _,train_acc,train_bleu_score,train_predict_list,train_target_list,train_source_list = calc_test_loss(Dataset(train_x[::100],train_y[::100]),display=False)
            predict_texts = get_all_text(predict_list)
            target_texts = get_all_text(target_list)
            source_texts = get_all_en_text(source_list)
            
            train_predict_texts = get_all_text(train_predict_list)
            train_target_texts = get_all_text(train_target_list)
            train_source_texts = get_all_en_text(train_source_list)
            
            with open('eval/{}/{}_{}_predict'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in predict_texts:
                    whdl.write("{}\n".format(line))
            with open('eval/{}/{}_{}_target'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in target_texts:
                    whdl.write("{}\n".format(line))
            with open('eval/{}/{}_{}_source'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in source_texts:
                    whdl.write("{}\n".format(line))
                    
            with open('eval/{}/{}_{}_predict_train'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in train_predict_texts:
                    whdl.write("{}\n".format(line))
            with open('eval/{}/{}_{}_target_train'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in train_target_texts:
                    whdl.write("{}\n".format(line))
            with open('eval/{}/{}_{}_source_train'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in train_source_texts:
                    whdl.write("{}\n".format(line))
            print("\niter {} step {} train loss {} train acc {} test loss {} test acc {} bleu {} lr {}\n".format(i+1,j,np.average(train_loss_list[-val_step:]),train_acc,test_loss,test_acc,bleu_score,lr))
            try:
                saver = tf.train.Saver()
                saver.save(session,'middleresult/{}/result_{}_{}'.format(model_path,i + 1,j))
            except:
                print('save fail')
        lr_step = int(worksum / 2) - 1
        if j % lr_step == 0 and j != 0:
            if (i + 1) >= 6:
                lr = lr / 2
        pb.complete(1)

iter 5 loss:51.32240676879883 lr:1 37.23 % [==================>--------------------------------] 11087/29780 	 used:98s eta:165 s

In [62]:
session

In [69]:
len([ind2en.get(i,'') for i in test_x[1]])

35

# try to translate

In [175]:
sent = 'do you know where is the nearest shopping center ?'.split()

In [176]:
sent

['do',
 'you',
 'know',
 'where',
 'is',
 'the',
 'nearest',
 'shopping',
 'center',
 '?']

In [177]:
sents = [en2ind.get(i) for i in sent]

In [178]:
sents = tf.contrib.keras.preprocessing.sequence.pad_sequences([sents],seq_max_len,padding='post')

In [179]:
sents

array([[  62,   16,  101,  129,   12,    2, 6984, 2924,  858,   39,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0]])

In [180]:
np.repeat(sents,35,axis=0)

array([[ 62,  16, 101, ...,   0,   0,   0],
       [ 62,  16, 101, ...,   0,   0,   0],
       [ 62,  16, 101, ...,   0,   0,   0],
       ..., 
       [ 62,  16, 101, ...,   0,   0,   0],
       [ 62,  16, 101, ...,   0,   0,   0],
       [ 62,  16, 101, ...,   0,   0,   0]])

In [181]:
sum(sents[0] > 0) + 1

11

In [182]:
tran = session.run([translations],feed_dict={x:np.repeat(sents,64,axis=0),x_len:[35] * 64, x_real_len:[sum(sents[0] > 0) + 1] * 64})

In [183]:
[ind2ch.get(i,'') for i in tran[0][0]]

['你', '知道', '最近', '的', '购物中心', '哪里', '吗', '？', '', '', '']

# release the model

In [160]:
!mkdir release

In [163]:
os.mkdir('release/align_and_translate_char')

In [165]:
saver.save(session,'release/align_and_translate_char/align_and_translate_model')

'release/align_and_translate/align_and_translate_model'